In [1]:
#sparting spark and reading Reddit
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Reddit").getOrCreate()
df = None
df = spark.read.csv("s3a://andrew-bierbaum-insight-test-dataset/Reddit/Reddit_Comments_2011*.csv.gz", header=True,multiLine=True, escape='"')


In [2]:

#showing the start of the data and format
df.show(5)
df.printSchema()

+--------------------+------------+--------+----------+---------+-----------------+-----+-----------+------------+--------+----------+-----+------------+----------------+------+-------+---------+---+-------------+----------------------+--------------+
|                body|score_hidden|archived|      name|   author|author_flair_text|downs|created_utc|subreddit_id| link_id| parent_id|score|retrieved_on|controversiality|gilded|     id|subreddit|ups|distinguished|author_flair_css_class|removal_reason|
+--------------------+------------+--------+----------+---------+-----------------+-----+-----------+------------+--------+----------+-----+------------+----------------+------+-------+---------+---+-------------+----------------------+--------------+
|Richtig, es geht ...|       false|    true|t1_c35plz4|[deleted]|             null|    0| 1323161654|     t5_22i0|t3_n1bte|t1_c35hgrf|    0|  1427980714|               0|     0|c35plz4|       de|  0|         null|                  null|        

In [3]:
# #Convert spark data to be readable using sql queries and query for the keyword terms
df.createOrReplaceTempView("Reddit")
xamarin_results = spark.sql("SELECT created_utc,link_id,  body FROM Reddit WHERE body RLIKE 'xamarin|Xamarin'")
flutter_results = spark.sql("SELECT created_utc, body FROM Reddit WHERE body RLIKE 'flutter|Flutter'")
react_native_results = spark.sql("SELECT created_utc, body FROM Reddit WHERE body RLIKE 'react native|React native'")

In [4]:
#Test output
flutter_results.show()

+-----------+--------------------+
|created_utc|                body|
+-----------+--------------------+
| 1315113994|As much as I enjo...|
| 1311721933|I remember ferven...|
| 1317357839|Not quite flutter...|
| 1297061315|Pick someone. Ask...|
| 1320253338|We'd had an amazi...|
| 1325283492|S'okay, we've spa...|
| 1310041003|I don't agree wit...|
| 1323901634|I'm sure Flutters...|
| 1324962558|[](/b02 "Where am...|
| 1313910798|http://flankbook....|
| 1323169614|Sadly dash isnt o...|
| 1315631627|Yes it is.  I hap...|
| 1310175884|As soon as the ca...|
| 1318207656|[](/flutterblush ...|
| 1306376707|My order: Rainbow...|
| 1322238370|[](/flutteryay "I...|
| 1322787183|[](/fluttershh "B...|
| 1325231493|[](/loveme) tirin...|
| 1324230053|Also, you can nev...|
| 1319309243|[](/flutterwink) ...|
+-----------+--------------------+
only showing top 20 rows



In [5]:
#collect, convert dates to datetime format for later graphing, and sort data
from datetime import datetime
python_flutter_results = flutter_results.collect()
python_flutter_results_cleaned = [(datetime.fromtimestamp(float(date)),body.encode('ascii',errors='ignore')) for date, body in python_flutter_results]
python_flutter_results_cleaned.sort()

python_xamarin_results = xamarin_results.collect()
python_xamarin_results_cleaned = [(datetime.fromtimestamp(float(date)), body.encode('ascii',errors='ignore')) for date, body in python_xamarin_results]
python_xamarin_results_cleaned.sort()

python_react_native_results = react_native_results.collect()
python_react_native_results_cleaned = [(datetime.fromtimestamp(float(date)),body.encode('ascii',errors='ignore')) for date, body in python_react_native_results]
python_react_native_results_cleaned.sort()

ValueError: too many values to unpack

In [ ]:
#sort, number, and then graph the data
%matplotlib inline
import numpy
import matplotlib
import matplotlib.pyplot as plt
import pandas
# import sqlite3

# conn = sqlite3.connect('Reddit.db')
# cur = conn.cursor()
# DATASET_COLUMNS = ["count","body","date"]
# cur.execute("CREATE TABLE IF NOT EXISTS reddit (date FLOAT PRIMARY KEY, body , date DATETIME);")
# cur.execute("CREATE TABLE twitter (target, ids, date, flag, user,text);")

# #https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html
# #https://www.kaggle.com/kazanova/sentiment140/version/2
# df = pandas.read_csv('TwitterSmall.csv', encoding='ISO-8859-1', names=DATASET_COLUMNS)

#testing to see if loading is working
#print(len(df))
#print(df.head(5))
df.to_sql('twitter', conn, if_exists='replace', index=False)
#don't rebuild the table if it already exists
cur.execute("SELECT count(*) FROM sqlite_master WHERE type ='table'")
if cur.fetchall()[0] == 0:
    cur.execute("CREATE TABLE twitter (target, ids, date, flag, user,text);")
    #read the csv  
    df = pandas.read_csv('TwitterSmall.csv', encoding='ISO-8859-1', names=DATASET_COLUMNS)
    #load into sql
    df.to_sql('twitter', conn, if_exists='replace', index=False)

#do SQL query and fetch results
cur.execute("SELECT date FROM twitter WHERE text LIKE '%c++%'")
times = cur.fetchall()
cleantime = [i[0] for i in times]

#clean data (data,) - has a useless touple & produce a count
cleantime = [i[0] for i in times]
#print(times)
count = numpy.arange(len(cleantime))

In [ ]:
#graphing and later csv export for flutter
count = numpy.arange(len(python_flutter_results_cleaned))
Date_Data = []
Body_Data = []
for date, body in python_flutter_results_cleaned:
    Date_Data.append(date)
    Body_Data.append(body)
matplotlib.pyplot.plot_date(Date_Data,count,xdate=True, drawstyle = 'steps-pre', linestyle = 'solid' )
matplotlib.pyplot.ylabel('Flutter Mentions')
matplotlib.pyplot.title('Reddit Flutter mentions')
plt.show()
plt.savefig('Reddit_flutter.png')

#print csv for flutter
pandas_df = pandas.DataFrame({'date':Date_Data,'body':Body_Data})
with open('Reddit_flutter.csv', 'a') as f:
    pandas_df.to_csv(f, header=False)

In [ ]:
#graphing and later csv export for xamarin
count = numpy.arange(len(python_xamarin_results_cleaned))
Date_Data = []
Body_Data = []
for date, body in python_xamarin_results_cleaned:
    Date_Data.append(date)
    Body_Data.append(body)
matplotlib.pyplot.plot_date(Date_Data,count,xdate=True, drawstyle = 'steps-pre', linestyle = 'solid' )
matplotlib.pyplot.ylabel('Xamarin Mentions')
matplotlib.pyplot.title('Reddit Xamarin mentions')
plt.show()
plt.savefig('Reddit_xamarin.png')

#print csv for xamarin
pandas_df = pandas.DataFrame({'date':Date_Data,'body':Body_Data})
with open('Reddit_xamarin.csv', 'a') as f:
    pandas_df.to_csv(f, header=False)

In [ ]:
#repeat graphing and later csv export for react native
count = numpy.arange(len(python_react_native_results_cleaned))
Date_Data = []
Body_Data = []
for date, body in python_react_native_results_cleaned:
    Date_Data.append(date)
    Body_Data.append(body)
matplotlib.pyplot.plot_date(Date_Data,count,xdate=True, drawstyle = 'steps-pre', linestyle = 'solid' )
matplotlib.pyplot.ylabel('React Native Mentions')
matplotlib.pyplot.title('Reddit React Native mentions')
plt.show()
plt.savefig('Reddit_react_native.png')

#print csv for react native
pandas_df = pandas.DataFrame({'date':Date_Data,'body':Body_Data})
with open('Reddit_react_native.csv', 'a') as f:
    pandas_df.to_csv(f, header=False)